In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns


In [69]:
df= pd.read_csv("../input/mechanical-properties-of-low-alloy-steels/MatNavi Mechanical properties of low-alloy steels.csv")

In [70]:
#first 10 rows of the dataset
df.head(10)

In [71]:
#last 10 rows of the dataset
df.tail(10)

In [72]:
#checking number of rows and columns
df.shape

In [73]:
df.info()

#except Alloy code all are numerical data

In [74]:
#Checking statistical description of our dataset
df.describe()

In [75]:
#checking null/missing values
[features for features in df.columns if df[features].isnull().sum()>0]

In [76]:
#Names of the columns in our dataset
columns=df.columns
print("List of Columns \n", columns)

In [77]:
#checking the most 
df['Alloy code'].value_counts()

In [78]:
#Checking for numboer of unique alloy codes in our dataset
alloy_names=df['Alloy code'].unique()
print("List of alloy codes \n",alloy_names)
total_alloy_number= df['Alloy code'].nunique()
print("Total Number of Alloy codes \n",total_alloy_number)

In [79]:
#label encoding to change categorical values into numerical values
from sklearn.preprocessing import LabelEncoder
lb= LabelEncoder().fit_transform(df['Alloy code'])
print(lb)

In [80]:
#Creatting a new DataFrame with the encoded data
df1=pd.DataFrame(lb)
df1

In [81]:
#Merging the and combining encoded data frame with our original dataset
df2 = pd.concat([df1, df], axis=1)
df2

In [82]:
#dropping the column with categorical data
df3=df2.drop(['Alloy code'], axis=1)

In [83]:
#renaming the column name of our encoded value column 
df3.rename(columns={0: "Alloy code"}, inplace=True)

In [84]:
df3.head()

In [85]:
#Seperating dependent and independent features for model building
X=df3.iloc[:,1:-4]
y=df3.iloc[:,-4:]

In [86]:
y.head()

In [87]:
X.head()

In [88]:
#checking the maximum values for independent features
y.apply(np.max)

In [89]:
corelation = df3.corr()
plt.subplots(figsize=(15,10))
sns.heatmap(corelation, xticklabels=corelation.columns, yticklabels=corelation.columns, annot=True)

for col in df3:
    plt.figure(figsize=(24,6))
    sns.scatterplot(x=X[col],y=y[' 0.2% Proof Stress (MPa)'])
    plt.title(f"Relation between {col} and ' 0.2% Proof Stress (MPa)'")

In [90]:
plt.figure(figsize=(12,5))
plt.subplot(1,5,1)
sns.distplot(df[' C'])
plt.subplot(1,5,2)
sns.distplot(df[' Si'])
plt.subplot(1,5,3)
sns.distplot(df[' Mn'])
plt.subplot(1,5,4)
sns.distplot(df[' P'])
plt.subplot(1,5,5)
sns.distplot(df[' Ni'])
plt.show()

In [91]:
plt.figure(figsize=(9, 8))
sns.distplot(df3[' 0.2% Proof Stress (MPa)'], color='b', bins=100, hist_kws={'alpha': 0.4});

In [92]:
plt.figure(figsize=(9, 8))
sns.distplot(df3[' Tensile Strength (MPa)'], color='b', bins=100, hist_kws={'alpha': 0.4});

In [93]:
plt.figure(figsize=(9, 8))
sns.distplot(df3[' Reduction in Area (%)'], color='b', bins=100, hist_kws={'alpha': 0.4});

We can see that the prices are skewed left and some outliers lies below 30.

In [94]:
plt.figure(figsize=(9, 8))
sns.distplot(df3[' Elongation (%)'], color='b', bins=100, hist_kws={'alpha': 0.4});

We can see that the prices are skewed right and some outliers lies above ~55

In [95]:
df3.columns

In [96]:
#loading all the numerical data
num_data = df3.iloc[:,1:]
num_data.head()

In [97]:
#visualizing the numerical values of our dataset
num_data.hist(figsize=(15, 15), bins=50, xlabelsize=8, ylabelsize=8);

As we already know from theoritical aspect, the elongation and reduction in area is inversely distributed.

In [98]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
y = sc.fit_transform(y)
X = sc.fit_transform(X)

In [99]:
y

In [100]:
X

In [101]:
#splitting our dataset for model building
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

 # **Linear Regression Model**

In [102]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

lr.fit(X_train, y_train)

In [103]:
lr_pred=lr.predict(X_test)
test_data_accuracy_LR= r2_score(lr_pred,y_test)
print("Accuracy using Linear Regression model is : ",test_data_accuracy_LR)

# **Decision Tree Model**

In [104]:
from sklearn.tree import DecisionTreeRegressor
dt=DecisionTreeRegressor()

dt.fit(X_train, y_train)

In [105]:
dt_pred= dt.predict(X_test)
test_data_accuracy_DT= r2_score(dt_pred,y_test)
print("Accuracy using Decision Treee model is : ",test_data_accuracy_DT)

# **Random Forest**

In [106]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()

rf.fit(X_train, y_train)

In [107]:
rf_pred=rf.predict(X_test)
test_data_accuracy_RF= r2_score(rf_pred,y_test)
print("Accuracy using Random Forest model is : ",test_data_accuracy_RF)

# **KNNeighbors**

In [108]:
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor()

knn.fit(X_train,y_train)

In [109]:
knn_pred=knn.predict(X_test)
test_data_accuracy_KNN= r2_score(knn_pred,y_test)
print("Accuracy using KNeighbors Regression model is : ",test_data_accuracy_KNN)

Random Forest Regressor gives the highest accuracy among other Regression algorithms

In [110]:
from sklearn.metrics import mean_absolute_error as mbe , mean_squared_error as mae

In [111]:
#Calculating the error loss
error_loss=mbe(rf_pred,y_test)
squared_error_loss= mae(rf_pred, y_test)
print("Mean Squared Error is ", squared_error_loss)
print("Mean Absolute Error is ", error_loss)

In [112]:
#Creating a dataframe using the accuracy received from different models
data= {'Model': ('LR','DT','RF','KNN'),
    'Accuracy' : (test_data_accuracy_LR,test_data_accuracy_DT,test_data_accuracy_RF, test_data_accuracy_KNN)
      }

In [113]:
df5 = pd.DataFrame(data)
df5.head()
  

In [115]:
df5.plot(x='Model', y='Accuracy', xlabel='Regression Models', ylabel='Accuracy',linestyle='dashed', color='magenta',marker='o')
plt.show()

The random forest regressor gives higher accuracy as compared to other regression models. Hence, this model makes an ideal choice for prediction of mechanical properties of low-alloy steels with R2 score of 89.7% which is significantly greater than R² score of other popularly known regression models.

Also error received for this model is small, which means we have trained a good model
* Mean Squared Error as  0.07874864369089093
* Mean Absolute Error as  0.16816265199480868

For a detailed case study check out my GitHub : https://github.com/skt-shreyas